Import yearly cleanup script

In [1]:
import yearly_cleanup as yc

- Input your cleanup year
- Create the new yearly folder

In [2]:
year = yc.get_year()
yc.create_year_folder(year)

Creating folder for year: 2024

Created folder:


'../site/releases/2024/'

Retrieve all the subdirectories matching your cleanup year

In [3]:
yc.get_yearly_releases(year)

Found 13 release folders for year 2024:


['2024-aug-13',
 '2024-dec-06',
 '2024-dec-24',
 '2024-feb-14',
 '2024-jan-18',
 '2024-jan-26',
 '2024-jul-22',
 '2024-jun-10',
 '2024-mar-27',
 '2024-may-22',
 '2024-oct-22',
 '2024-sep-09',
 '2024-sep-25']